<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/MACD_48.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from tqdm import tqdm

bist30_tickers = [
    'AEFES.IS', 'AKBNK.IS', 'AKSEN.IS', 'ARCLK.IS', 'ASELS.IS',
    'BIMAS.IS', 'DOHOL.IS', 'EKGYO.IS', 'EREGL.IS', 'FROTO.IS',
    'GARAN.IS', 'GUBRF.IS', 'HEKTS.IS', 'ISCTR.IS', 'KRDMD.IS',
    'KCHOL.IS', 'KOZAL.IS', 'MPARK.IS', 'MGROS.IS', 'PETKM.IS',
    'PGSUS.IS', 'SAHOL.IS', 'SASA.IS', 'SISE.IS', 'TAVHL.IS',
    'TOASO.IS', 'TTKOM.IS', 'TUPRS.IS', 'VAKBN.IS', 'YKBNK.IS'
]

def calculate_macd(data, ema_s=12, ema_l=26, signal_mw=9):
    data["EMA_S"] = data['Close'].ewm(span=ema_s, min_periods=ema_s).mean()
    data["EMA_L"] = data['Close'].ewm(span=ema_l, min_periods=ema_l).mean()
    data["MACD"] = data["EMA_S"] - data["EMA_L"]
    data["MACD_Signal"] = data["MACD"].ewm(span=signal_mw, min_periods=signal_mw).mean()
    data["MACD_Hist"] = data["MACD"] - data["MACD_Signal"]
    return data

def check_macd_crossover(data):
    # MACD ve sinyal çizgisi kesişimlerini kontrol et
    data["MACD_Crossover"] = np.where(
        (data["MACD"].shift(1) < data["MACD_Signal"].shift(1)) & (data["MACD"] > data["MACD_Signal"]),
        "Bullish",
        np.where(
            (data["MACD"].shift(1) > data["MACD_Signal"].shift(1)) & (data["MACD"] < data["MACD_Signal"]),
            "Bearish",
            None
        )
    )
    return data

results = []
for ticker in tqdm(bist30_tickers, desc="BIST 30 Taraması"):
    try:
        data = yf.download(ticker, start="2023-01-01", end="2024-11-29")
        data = calculate_macd(data)
        data = check_macd_crossover(data)

        if data["MACD_Crossover"].iloc[-1] in ["Bullish", "Bearish"]:
            results.append({
                "Ticker": ticker,
                "Date": data.index[-1],
                "Crossover": data["MACD_Crossover"].iloc[-1]
            })
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

results_df = pd.DataFrame(results)
if not results_df.empty:
    print("Son Gün MACD Kesişimleri:")
    print(results_df)
else:
    print("Son gün için herhangi bir kesişim bulunamadı.")